# 病床姿勢監測 - 語義引導熱成像超解析度訓練（完整版）

**專案**: 熱成像超解析度 + 姿勢辨識  
**作者**: rochi190  
**日期**: 2025-11-03  

## 訓練策略
```
階段 1: RGB 人體分割 (UNet)
  RGB (高解析度) → Segmentation UNet → Person Mask

階段 2: 語義引導超解析度 (Guided SR)
  輸入: Thermal LR (32×24) + Person Mask (高解析度)
  輸出: Thermal HR (256×256)
  Loss: L1 Loss + Perceptual Loss + Person Region Focus Loss

階段 3: 姿勢辨識 (Pose Classifier)
  Thermal HR → CNN → Pose Class (lying/sitting/fallen)
```

---
## Step 0: 環境設定

In [ ]:
import os
import sys
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from tqdm import tqdm
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.utils.tensorboard import SummaryWriter

# Matplotlib
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 隨機種子
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("="*70)
print("病床姿勢監測系統 - 訓練模組")
print("="*70)
print(f"執行時間: {pd.Timestamp.now()}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"訓練裝置: {device}")
print("="*70)

---
## Step 1: 路徑與參數配置

In [ ]:
# 路徑
BASE_DIR = Path(os.getcwd())
OUTPUT_DIR = BASE_DIR / 'output'

TRAINING_DIR = OUTPUT_DIR / 'training_dataset'
ADAPTIVE_DIR = OUTPUT_DIR / 'adaptive_pairing'
MODELS_DIR = OUTPUT_DIR / 'trained_models'
LOGS_DIR = OUTPUT_DIR / 'training_logs'
RESULTS_DIR = OUTPUT_DIR / 'training_results'

for d in [MODELS_DIR, LOGS_DIR, RESULTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# 資料
LABELS_CSV = TRAINING_DIR / 'pose_labels_with_thermal.csv'
MASKS_DIR = TRAINING_DIR / 'person_masks'
PAIRS_JSON = ADAPTIVE_DIR / 'pairs_mapping.json'

# 訓練參數
STAGE1_CONFIG = {
    'name': 'RGB_Segmentation',
    'epochs': 50,
    'batch_size': 16,
    'learning_rate': 1e-4,
    'input_size': (256, 256),
}

STAGE2_CONFIG = {
    'name': 'Semantic_Guided_SR',
    'epochs': 100,
    'batch_size': 32,
    'learning_rate': 1e-4,
    'thermal_input_size': (24, 32),
    'thermal_output_size': (256, 256),
    'scale_factor': 8,
    'lambda_l1': 1.0,
    'lambda_perceptual': 0.1,
    'lambda_person_focus': 0.5,
}

# 階段 3: 姿勢辨識配置
STAGE3_CONFIG = {
    'name': 'Pose_Classifier',
    'epochs': 50,
    'batch_size': 32,
    'learning_rate': 1e-3,
    'num_classes': 3,  # 🔧 改為 3 類（lying, sitting, fallen）
}

TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.1
TEST_SPLIT = 0.1

print(f"路徑檢查:")
print(f"  Labels: {'✓' if LABELS_CSV.exists() else '✗'} {LABELS_CSV}")
print(f"  Masks: {'✓' if MASKS_DIR.exists() else '✗'} {MASKS_DIR}")
print(f"  Pairs: {'✓' if PAIRS_JSON.exists() else '✗'} {PAIRS_JSON}")

---
## Step 2: 準備訓練資料（從 Adaptive Pairing 提取）

In [ ]:
# 從 adaptive pairing 載入配對
with open(PAIRS_JSON, 'r') as f:
    pairs_data = json.load(f)

print(f"載入 {len(pairs_data)} 個配對")

# 需要先提取 Thermal 和 RGB 圖片並儲存為 .npy
# (這部分在 01 中已經完成，這裡假設資料已存在)

# 檢查是否已有 aligned_dataset
ALIGNED_DIR = OUTPUT_DIR / 'aligned_dataset'
THERMAL_DIR = ALIGNED_DIR / 'thermal'
RGB_DIR = ALIGNED_DIR / 'rgb'

if not THERMAL_DIR.exists() or not RGB_DIR.exists():
    print("\n⚠️ 警告: aligned_dataset 不存在")
    print("請先執行以下步驟提取資料:")
    print("1. 從 thermal log 重新載入圖片")
    print("2. 從 AVI 提取對應的 RGB 幀")
    print("3. 儲存為 .npy 檔案")
    raise FileNotFoundError("請先準備 aligned_dataset")

print(f"\nAligned dataset:")
print(f"  Thermal: {THERMAL_DIR}")
print(f"  RGB: {RGB_DIR}")

---
## Step 3: 資料集類別定義

In [ ]:
class RGBSegmentationDataset(Dataset):
    def __init__(self, labels_df, rgb_dir, masks_dir, input_size=(256, 256)):
        self.labels_df = labels_df
        self.rgb_dir = rgb_dir
        self.masks_dir = masks_dir
        self.input_size = input_size
    
    def __len__(self):
        return len(self.labels_df)
    
    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]
        
        # RGB
        rgb_path = self.rgb_dir / f"{row['pair_id']}_rgb.npy"
        rgb = np.load(rgb_path)
        rgb = cv2.resize(rgb, self.input_size)
        rgb = rgb.astype(np.float32) / 255.0
        
        # Mask
        mask_path = self.masks_dir / row['mask_file']
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, self.input_size, interpolation=cv2.INTER_NEAREST)
        mask = (mask > 127).astype(np.float32)
        
        rgb = torch.from_numpy(rgb).permute(2, 0, 1)
        mask = torch.from_numpy(mask).unsqueeze(0)
        
        return rgb, mask


class SemanticGuidedSRDataset(Dataset):
    def __init__(self, labels_df, thermal_dir, rgb_dir, masks_dir, 
                 thermal_input_size=(24, 32), output_size=(256, 256)):
        self.labels_df = labels_df
        self.thermal_dir = thermal_dir
        self.rgb_dir = rgb_dir
        self.masks_dir = masks_dir
        self.thermal_input_size = thermal_input_size
        self.output_size = output_size
    
    def __len__(self):
        return len(self.labels_df)
    
    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]
        
        # Thermal LR
        thermal_path = self.thermal_dir / f"{row['pair_id']}_thermal.npy"
        thermal_lr = np.load(thermal_path)
        thermal_lr = cv2.resize(thermal_lr, (self.thermal_input_size[1], self.thermal_input_size[0]))
        thermal_lr = thermal_lr.astype(np.float32) / 255.0
        
        # RGB (GT)
        rgb_path = self.rgb_dir / f"{row['pair_id']}_rgb.npy"
        rgb = np.load(rgb_path)
        rgb_gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
        rgb_gray = cv2.resize(rgb_gray, self.output_size)
        rgb_gray = rgb_gray.astype(np.float32) / 255.0
        
        # Person Mask
        mask_path = self.masks_dir / row['mask_file']
        person_mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        person_mask = cv2.resize(person_mask, self.output_size, interpolation=cv2.INTER_NEAREST)
        person_mask = (person_mask > 127).astype(np.float32)
        
        thermal_lr = torch.from_numpy(thermal_lr).unsqueeze(0)
        rgb_gray = torch.from_numpy(rgb_gray).unsqueeze(0)
        person_mask = torch.from_numpy(person_mask).unsqueeze(0)
        
        return {
            'thermal_lr': thermal_lr,
            'person_mask': person_mask,
            'rgb_gray_hr': rgb_gray,
            'pair_id': row['pair_id']
        }


class PoseClassificationDataset(Dataset):
    def __init__(self, labels_df, thermal_hr_dir, input_size=(256, 256)):
        # 🔧 只保留 3 個有效姿勢
        valid_poses = ['lying', 'sitting', 'fallen']
        self.labels_df = labels_df[labels_df['pose_label'].isin(valid_poses)].reset_index(drop=True)
        
        self.thermal_hr_dir = thermal_hr_dir
        self.input_size = input_size
        
        # 🔧 類別映射（3 類）
        self.class_to_idx = {
            'lying': 0, 
            'sitting': 1, 
            'fallen': 2
        }
    
    def __len__(self):
        return len(self.labels_df)
    
    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]
        
        thermal_hr_path = self.thermal_hr_dir / f"{row['pair_id']}_thermal_hr.npy"
        thermal_hr = np.load(thermal_hr_path)
        thermal_hr = cv2.resize(thermal_hr, self.input_size)
        thermal_hr = thermal_hr.astype(np.float32) / 255.0
        thermal_hr = torch.from_numpy(thermal_hr).unsqueeze(0)
        
        label = self.class_to_idx[row['pose_label']]
        
        return thermal_hr, label

print("✅ 資料集定義完成")

---
## Step 4: 模型架構定義

In [ ]:
# UNet 基礎模組
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.double_conv(x)


# 階段 1: UNet
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super().__init__()
        
        self.enc1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        
        self.bottleneck = DoubleConv(512, 1024)
        
        self.upconv4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = DoubleConv(128, 64)
        
        self.out_conv = nn.Conv2d(64, out_channels, 1)
    
    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool1(enc1))
        enc3 = self.enc3(self.pool2(enc2))
        enc4 = self.enc4(self.pool3(enc3))
        
        bottleneck = self.bottleneck(self.pool4(enc4))
        
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat([dec4, enc4], dim=1)
        dec4 = self.dec4(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat([dec3, enc3], dim=1)
        dec3 = self.dec3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat([dec2, enc2], dim=1)
        dec2 = self.dec2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat([dec1, enc1], dim=1)
        dec1 = self.dec1(dec1)
        
        return torch.sigmoid(self.out_conv(dec1))


# 階段 2: Semantic-Guided SR
class SemanticGuidedSRNet(nn.Module):
    def __init__(self, scale_factor=8):
        super().__init__()
        self.scale_factor = scale_factor
        
        self.thermal_encoder = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        
        self.mask_encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        
        self.fusion = nn.Sequential(
            nn.Conv2d(128 + 64, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        
        self.upsample = nn.Sequential(
            nn.Conv2d(128, 128 * (scale_factor ** 2), 3, padding=1),
            nn.PixelShuffle(scale_factor),
            nn.ReLU(inplace=True),
        )
        
        self.refine = nn.Sequential(
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 1, 3, padding=1),
            nn.Sigmoid()
        )
    
    def forward(self, thermal_lr, person_mask):
        thermal_feat = self.thermal_encoder(thermal_lr)
        thermal_feat_up = F.interpolate(thermal_feat, size=person_mask.shape[-2:], 
                                       mode='bilinear', align_corners=False)
        
        mask_feat = self.mask_encoder(person_mask)
        
        fused = torch.cat([thermal_feat_up, mask_feat], dim=1)
        fused = self.fusion(fused)
        
        upsampled = self.upsample(F.interpolate(thermal_feat, scale_factor=self.scale_factor, mode='nearest'))
        upsampled = F.interpolate(upsampled, size=person_mask.shape[-2:], mode='bilinear', align_corners=False)
        
        output = self.refine(upsampled)
        
        return output


# 階段 3: Pose Classifier
class PoseClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

print("✅ 模型定義完成")

---
## Step 5: Loss 函數

In [ ]:
class PerceptualLoss(nn.Module):
    def __init__(self):
        super().__init__()
        vgg = models.vgg16(pretrained=True).features[:16].eval()
        for param in vgg.parameters():
            param.requires_grad = False
        self.vgg = vgg
        self.criterion = nn.L1Loss()
    
    def forward(self, pred, target):
        pred_rgb = pred.repeat(1, 3, 1, 1)
        target_rgb = target.repeat(1, 3, 1, 1)
        
        pred_feat = self.vgg(pred_rgb)
        target_feat = self.vgg(target_rgb)
        
        return self.criterion(pred_feat, target_feat)


class PersonFocusLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.criterion = nn.L1Loss(reduction='none')
    
    def forward(self, pred, target, person_mask):
        loss = self.criterion(pred, target)
        weight = person_mask * 2.0 + 1.0
        weighted_loss = loss * weight
        return weighted_loss.mean()


class CombinedSRLoss(nn.Module):
    def __init__(self, lambda_l1=1.0, lambda_perceptual=0.1, lambda_person_focus=0.5):
        super().__init__()
        self.lambda_l1 = lambda_l1
        self.lambda_perceptual = lambda_perceptual
        self.lambda_person_focus = lambda_person_focus
        
        self.l1_loss = nn.L1Loss()
        self.perceptual_loss = PerceptualLoss()
        self.person_focus_loss = PersonFocusLoss()
    
    def forward(self, pred, target, person_mask):
        l1 = self.l1_loss(pred, target)
        perceptual = self.perceptual_loss(pred, target)
        person_focus = self.person_focus_loss(pred, target, person_mask)
        
        total_loss = (
            self.lambda_l1 * l1 +
            self.lambda_perceptual * perceptual +
            self.lambda_person_focus * person_focus
        )
        
        return total_loss, {
            'l1': l1.item(),
            'perceptual': perceptual.item(),
            'person_focus': person_focus.item(),
            'total': total_loss.item()
        }

print("✅ Loss 函數完成")

---
## Step 6: 載入並分割資料

In [ ]:
# 載入標籤
labels_df = pd.read_csv(LABELS_CSV)

print(f"\n總樣本數: {len(labels_df):,}")
print(f"\n姿勢分佈:")
print(labels_df['pose_label'].value_counts())

# 分割資料集
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(
    labels_df, test_size=TEST_SPLIT, random_state=SEED,
    stratify=labels_df['pose_label']
)

train_df, val_df = train_test_split(
    train_val_df, test_size=VAL_SPLIT/(TRAIN_SPLIT+VAL_SPLIT),
    random_state=SEED, stratify=train_val_df['pose_label']
)

print(f"\n資料分割:")
print(f"  訓練集: {len(train_df):,}")
print(f"  驗證集: {len(val_df):,}")
print(f"  測試集: {len(test_df):,}")

---
## Step 7: 階段 1 - 訓練 RGB 人體分割

In [ ]:
def train_stage1_segmentation(train_df, val_df, config, device):
    print(f"\n{'='*70}")
    print(f"階段 1: RGB 人體分割訓練")
    print(f"{'='*70}")
    
    train_dataset = RGBSegmentationDataset(train_df, RGB_DIR, MASKS_DIR, config['input_size'])
    val_dataset = RGBSegmentationDataset(val_df, RGB_DIR, MASKS_DIR, config['input_size'])
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4)
    
    model = UNet().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5)
    
    writer = SummaryWriter(LOGS_DIR / config['name'])
    
    best_val_loss = float('inf')
    
    for epoch in range(config['epochs']):
        model.train()
        train_loss = 0
        
        for rgb, mask in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']} [Train]"):
            rgb, mask = rgb.to(device), mask.to(device)
            
            optimizer.zero_grad()
            pred = model(rgb)
            loss = criterion(pred, mask)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for rgb, mask in val_loader:
                rgb, mask = rgb.to(device), mask.to(device)
                pred = model(rgb)
                loss = criterion(pred, mask)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        
        print(f"Epoch {epoch+1}: Train={train_loss:.4f}, Val={val_loss:.4f}")
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), MODELS_DIR / f"{config['name']}_best.pth")
            print(f"  ✓ Best model saved")
    
    writer.close()
    print(f"\n✅ 階段 1 完成，Best Val Loss: {best_val_loss:.4f}")
    return model

# 執行
# segmentation_model = train_stage1_segmentation(train_df, val_df, STAGE1_CONFIG, device)

---
## Step 8: 階段 2 - 訓練語義引導超解析度

In [ ]:
def train_stage2_sr(train_df, val_df, config, device):
    print(f"\n{'='*70}")
    print(f"階段 2: 語義引導超解析度訓練")
    print(f"{'='*70}")
    
    train_dataset = SemanticGuidedSRDataset(
        train_df, THERMAL_DIR, RGB_DIR, MASKS_DIR,
        config['thermal_input_size'], config['thermal_output_size']
    )
    val_dataset = SemanticGuidedSRDataset(
        val_df, THERMAL_DIR, RGB_DIR, MASKS_DIR,
        config['thermal_input_size'], config['thermal_output_size']
    )
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4)
    
    model = SemanticGuidedSRNet(scale_factor=config['scale_factor']).to(device)
    criterion = CombinedSRLoss(
        lambda_l1=config['lambda_l1'],
        lambda_perceptual=config['lambda_perceptual'],
        lambda_person_focus=config['lambda_person_focus']
    ).to(device)
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'])
    
    writer = SummaryWriter(LOGS_DIR / config['name'])
    
    best_val_loss = float('inf')
    
    for epoch in range(config['epochs']):
        model.train()
        train_losses = defaultdict(float)
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']} [Train]"):
            thermal_lr = batch['thermal_lr'].to(device)
            person_mask = batch['person_mask'].to(device)
            rgb_gray_hr = batch['rgb_gray_hr'].to(device)
            
            optimizer.zero_grad()
            pred_hr = model(thermal_lr, person_mask)
            loss, loss_dict = criterion(pred_hr, rgb_gray_hr, person_mask)
            loss.backward()
            optimizer.step()
            
            for k, v in loss_dict.items():
                train_losses[k] += v
        
        for k in train_losses:
            train_losses[k] /= len(train_loader)
        
        model.eval()
        val_losses = defaultdict(float)
        
        with torch.no_grad():
            for batch in val_loader:
                thermal_lr = batch['thermal_lr'].to(device)
                person_mask = batch['person_mask'].to(device)
                rgb_gray_hr = batch['rgb_gray_hr'].to(device)
                
                pred_hr = model(thermal_lr, person_mask)
                _, loss_dict = criterion(pred_hr, rgb_gray_hr, person_mask)
                
                for k, v in loss_dict.items():
                    val_losses[k] += v
        
        for k in val_losses:
            val_losses[k] /= len(val_loader)
        
        for k in train_losses:
            writer.add_scalar(f'Loss/train_{k}', train_losses[k], epoch)
            writer.add_scalar(f'Loss/val_{k}', val_losses[k], epoch)
        
        print(f"Epoch {epoch+1}: Train Total={train_losses['total']:.4f}, Val Total={val_losses['total']:.4f}")
        
        scheduler.step()
        
        if val_losses['total'] < best_val_loss:
            best_val_loss = val_losses['total']
            torch.save(model.state_dict(), MODELS_DIR / f"{config['name']}_best.pth")
            print(f"  ✓ Best model saved")
    
    writer.close()
    print(f"\n✅ 階段 2 完成，Best Val Loss: {best_val_loss:.4f}")
    return model

# 執行
# sr_model = train_stage2_sr(train_df, val_df, STAGE2_CONFIG, device)

---
## Step 9: 生成高解析度熱成像（階段 3 準備）

In [ ]:
def generate_thermal_hr(labels_df, sr_model, output_dir, device):
    print(f"\n生成高解析度熱成像...")
    
    output_dir.mkdir(exist_ok=True)
    sr_model.eval()
    
    dataset = SemanticGuidedSRDataset(
        labels_df, THERMAL_DIR, RGB_DIR, MASKS_DIR,
        (24, 32), (256, 256)
    )
    loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="生成 Thermal HR"):
            thermal_lr = batch['thermal_lr'].to(device)
            person_mask = batch['person_mask'].to(device)
            
            thermal_hr = sr_model(thermal_lr, person_mask)
            
            for i, pair_id in enumerate(batch['pair_id']):
                hr_img = thermal_hr[i, 0].cpu().numpy()
                hr_img = (hr_img * 255).astype(np.uint8)
                
                save_path = output_dir / f"{pair_id}_thermal_hr.npy"
                np.save(save_path, hr_img)
    
    print(f"✅ 完成，儲存於 {output_dir}")

# 執行
# THERMAL_HR_DIR = ALIGNED_DIR / 'thermal_hr'
# generate_thermal_hr(labels_df, sr_model, THERMAL_HR_DIR, device)

---
## Step 10: 階段 3 - 訓練姿勢辨識

In [ ]:
def train_stage3_pose_classifier(train_df, val_df, config, thermal_hr_dir, device):
    print(f"\n{'='*70}")
    print(f"階段 3: 姿勢辨識訓練")
    print(f"{'='*70}")
    
    train_dataset = PoseClassificationDataset(train_df, thermal_hr_dir)
    val_dataset = PoseClassificationDataset(val_df, thermal_hr_dir)
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4)
    
    model = PoseClassifier(num_classes=config['num_classes']).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
    
    writer = SummaryWriter(LOGS_DIR / config['name'])
    
    best_val_acc = 0
    
    for epoch in range(config['epochs']):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for thermal_hr, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']} [Train]"):
            thermal_hr, labels = thermal_hr.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(thermal_hr)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        train_loss /= len(train_loader)
        train_acc = 100. * train_correct / train_total
        
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for thermal_hr, labels in val_loader:
                thermal_hr, labels = thermal_hr.to(device), labels.to(device)
                outputs = model(thermal_hr)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        val_loss /= len(val_loader)
        val_acc = 100. * val_correct / val_total
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        
        print(f"Epoch {epoch+1}: Train Acc={train_acc:.2f}%, Val Acc={val_acc:.2f}%")
        
        scheduler.step()
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), MODELS_DIR / f"{config['name']}_best.pth")
            print(f"  ✓ Best model saved")
    
    writer.close()
    print(f"\n✅ 階段 3 完成，Best Val Acc: {best_val_acc:.2f}%")
    return model

# 執行
# pose_model = train_stage3_pose_classifier(train_df, val_df, STAGE3_CONFIG, THERMAL_HR_DIR, device)

---
## Step 11: 完整訓練流程

In [ ]:
print(f"\n{'='*70}")
print(f"開始完整訓練流程")
print(f"{'='*70}")

# 階段 1
print("\n🔄 階段 1: RGB 人體分割...")
segmentation_model = train_stage1_segmentation(train_df, val_df, STAGE1_CONFIG, device)

# 階段 2
print("\n🔄 階段 2: 語義引導超解析度...")
sr_model = train_stage2_sr(train_df, val_df, STAGE2_CONFIG, device)

# 生成 Thermal HR
THERMAL_HR_DIR = ALIGNED_DIR / 'thermal_hr'
generate_thermal_hr(labels_df, sr_model, THERMAL_HR_DIR, device)

# 階段 3
print("\n🔄 階段 3: 姿勢辨識...")
pose_model = train_stage3_pose_classifier(train_df, val_df, STAGE3_CONFIG, THERMAL_HR_DIR, device)

print(f"\n{'='*70}")
print(f"✅ 所有階段訓練完成！")
print(f"{'='*70}")
print(f"\n訓練模型:")
print(f"  - {MODELS_DIR / 'RGB_Segmentation_best.pth'}")
print(f"  - {MODELS_DIR / 'Semantic_Guided_SR_best.pth'}")
print(f"  - {MODELS_DIR / 'Pose_Classifier_best.pth'}")
print(f"\nTensorBoard logs:")
print(f"  tensorboard --logdir={LOGS_DIR}")
print(f"\n{'='*70}")